In [18]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [22]:
for filename in os.listdir("../data/RawData/BLS"):
    print(filename.split(".")[0])

banana
beef
bread
chicken
coffee
eggs
electricity
flour
iceCream
rice
unleadedGasoline


In [20]:
def toQuarter(month):
    if month>=1 and month<=3:
        return 1
    elif month > 3 and month <=6:
        return 2
    elif month > 6 and month <= 9:
        return 3
    else: 
        return 4

In [21]:
def formatData(filename):
    data = pd.read_csv('../data/RawData/BLS/'+filename,usecols=["Year","Period","Value"])
    
    data.Period = data.Period.apply(lambda x: x[1:]).astype("int")
    data["Quarter"] = data.Period.apply(toQuarter)
    data = data.drop(columns=["Period"])
    data = data.groupby(["Year","Quarter"]).mean()
    return data

In [25]:
def combine(df,left_on_col = ["YEAR FORECAST MADE","QUARTER"]):
    returnFrame = df.copy()
    for filename in os.listdir("../data/RawData/BLS"):
        to_merge = formatData(filename).reset_index(level=[0,1])
        to_merge.rename(columns={"Value":filename.split(".")[0]},inplace=True)
        returnFrame = returnFrame.merge(to_merge,how="left",left_on=left_on_col,right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
        #returnFrame = returnFrame.merge(to_merge,how="left",left_on=left_on_col,right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
    returnFrame = returnFrame[returnFrame.columns.drop(returnFrame.filter(regex='drop').columns)]
    return returnFrame.drop(columns=["Year","Quarter"])
    

In [26]:
training = pd.read_csv("../data/TrainingData/fulldata.csv")
training.head()

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,bin_value_sum,TDIST,actual,ACTUAL_BIN,MAX,NUM_OF_MAX,ACTUAL_CONF,HIT,pred_average,pred_var
0,1992,1,1992,RealGDP,20,2,1992-20-1992-1,RealGDP-1992-20-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.5,2.0,0.0,0.0,1.00,0.2500
1,1992,1,1992,RealGDP,30,3,1992-30-1992-1,RealGDP-1992-30-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.6,1.0,0.6,1.0,3.30,0.3600
2,1992,1,1992,RealGDP,35,2,1992-35-1992-1,RealGDP-1992-35-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.5,1.0,0.1,0.0,2.10,1.2400
3,1992,1,1992,RealGDP,40,2,1992-40-1992-1,RealGDP-1992-40-1992-1,NaN,NaN,...,NaN,0.75,3.52244,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1992,1,1992,RealGDP,60,2,1992-60-1992-1,RealGDP-1992-60-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.6,1.0,0.0,0.0,1.15,0.4275


In [29]:
newTraining = combine(training)
newTraining.tail()
newTraining.to_csv("../data/TrainingData/trainingWithItems.csv",index=False)

In [30]:
newTraining

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,beef,bread,chicken,coffee,eggs,electricity,flour,iceCream,rice,unleadedGasoline
0,1992,1,1992,RealGDP,20,2,1992-20-1992-1,RealGDP-1992-20-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
1,1992,1,1992,RealGDP,30,3,1992-30-1992-1,RealGDP-1992-30-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
2,1992,1,1992,RealGDP,35,2,1992-35-1992-1,RealGDP-1992-35-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
3,1992,1,1992,RealGDP,40,2,1992-40-1992-1,RealGDP-1992-40-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
4,1992,1,1992,RealGDP,60,2,1992-60-1992-1,RealGDP-1992-60-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31041,2021,2,2024,Unemployment,586,2,2024-586-2021-2,Unemployment-2024-586-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31042,2021,2,2024,Unemployment,587,2,2024-587-2021-2,Unemployment-2024-587-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31043,2021,2,2024,Unemployment,588,1,2024-588-2021-2,Unemployment-2024-588-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31044,2021,2,2024,Unemployment,589,2,2024-589-2021-2,Unemployment-2024-589-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333


In [95]:
quarterFred= pd.read_csv("../data/CleanData/QuarterlyFred.csv",index_col=0)
quarterFred["DATE"] = quarterFred["DATE"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
quarterFred["Year"] = quarterFred["DATE"].apply(lambda x:x.year)
quarterFred["Month"] = quarterFred["DATE"].apply(lambda y:y.month)
quarterFred["Quarter"] = quarterFred["Month"].apply(toQuarter)
quarterFred.drop(columns=["Month"],inplace=True)
quarterFred.head()

,DATE,GDPC1_PCH,GNP_PCH,GNPC96_PCH,CPILFESL,PCEPILFE,JHDUSRGDPBR,UNRATE,Year,Quarter
0,1947-04-01,-0.26669,1.19643,-0.23978,NaN,NaN,NaN,NaN,1947,2
1,1947-07-01,-0.20650,1.47857,-0.20822,NaN,NaN,NaN,NaN,1947,3
2,1947-10-01,1.56421,4.09427,1.58636,NaN,NaN,NaN,NaN,1947,4
3,1948-01-01,1.50426,2.35726,1.54854,NaN,NaN,NaN,3.733333,1948,1
4,1948-04-01,1.65066,2.58785,1.67265,NaN,NaN,NaN,3.666667,1948,2


In [97]:
newQuarterly = combine(quarterFred,["Year","Quarter"])
newQuarterly.head()
#newQuarterly.to_csv("../data/CleanData/QuarterlyFredWithItems.csv",index=False)

,DATE,GDPC1_PCH,GNP_PCH,GNPC96_PCH,CPILFESL,PCEPILFE,JHDUSRGDPBR,UNRATE,banana,beef,bread,chicken,coffee,eggs,electricity,flour,rice,unleadedGasoline
0,1947-04-01,-0.26669,1.19643,-0.23978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947-07-01,-0.20650,1.47857,-0.20822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1947-10-01,1.56421,4.09427,1.58636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1948-01-01,1.50426,2.35726,1.54854,NaN,NaN,NaN,3.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1948-04-01,1.65066,2.58785,1.67265,NaN,NaN,NaN,3.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
